####  An Indian business man wants is looking forward to set up his business in India and USA and wants his team to perform data analysis based on the data gathered.

##### His questions are as follows -

- What is the distribution of eye color among people in the table?
- What is the average age of people in the table?
- What is the most common country of origin for people in the table?
- How many unique postal codes exist in the table?
- How many unique cities exist in the table ?
- What is the most common city of residence for people in the table?
- What is the most popular food among people in the table?
- How many unique foods do people in the table like?
- What is the average number of foods a person likes in the table?
- Are there any people who like the same food?
- What is the most common city of residence for people in the table?
- What is the most common country of origin for people in the table?
- What is the most common nationality among people in the table?
- How many unique countries do people in the table reside in?
- What is the average salary of employees in the table?
- What is the most common department among employees in the table?
- What is the average salary in each department?
- What is the most common company name in the table?
- What is the average salary of people living in a particular city or country?
- What is the most common eye color among people in a specific department of a company?
- Which country has the most number of people with a specific favorite food?
- What is the average age of people in a specific department of a company?
- How many people of different nationalities work for a specific company?
- Which city has the highest number of people with a specific eye color?
- What is the most common favorite food among people in a specific country?
- What is the average salary of people with a specific eye color in a specific city?
- How many people from a specific state work for a company located in a specific country?
- What is the average salary of people with a specific nationality working for a specific company?

Let's see the first dataset that is available which is called as person

In [0]:
create_person_df = spark.read \
                   .option("Inferschema",True) \
                   .option("header",True) \
                   .format("csv") \
                   .load('dbfs:/FileStore/person.csv')

Note that inferSchema option tells the reader to infer data types from the source file. This results in an additional pass over the file resulting in two Spark jobs being triggered. It is an expensive operation because Spark must automatically go through the CSV file and infer the schema for each column. In this case, we are using it because we know that the dataset is small.

In [0]:
display(create_person_df)

person_id,fname,lname,eye_color,dob,street,city,state,country,postal_code
1,Chandru,Rathod,BR,1995-02-23,H1 road,Mumbai,MH,India,23456
2,Raju,Desai,BL,1993-01-30,Udagiri,Bangalore,KA,India,22356
3,Rajendra,Diman,BL,1993-01-30,Kengeri,Madikeri,KA,India,21056
4,Rigan,Thomas,BL,1993-01-30,Udagiri,Hassan,KA,India,22056
5,John,Doe,BL,1999-01-01,123 Main St,San Francisco,California,USA,12345
6,Jane,Smith,GR,1998-02-14,456 Oak Ave,Los Angeles,California,USA,67890
7,Michael,Johnson,BR,1997-03-25,789 Maple St,New York,New York,USA,11111
8,Emily,Davis,BL,1996-04-16,111 Pine St,Seattle,Washington,USA,22222
9,William,Brown,BR,1995-05-27,222 Cedar Ave,Chicago,Illinois,USA,33333
10,Brittney,Thomas,BL,1990-10-12,777 Maple St,Phoenix,Arizona,USA,88888


Let's us start answering the first set of questions -
- What is the distribution of eye color among people in the table?
- What is the average age of people in the table?
- What is the most common country of origin for people in the table?
- How many unique postal codes exist in the table?
- How many unique cities are there in the table ?
- What is the most common city of residence for people in the table?

In [0]:
%sql
DROP TABLE if exists person

In [0]:
%sql
select * from person

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-1442428737857269> in <cell line: 1>()
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

<command-1442428737857269> in ____databricks_percent_sql()
      2   def ____databricks_percent_sql():
      3     import base64
----> 4     df = spark.sql(base64.standard_b64decode("c2VsZWN0ICogZnJvbSBwZXJzb24=").decode())
      5     display(df)
      6     return df

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/sp

In [0]:
create_person_df.write.mode("overwrite").option("mergeSchema","True").saveAsTable("person")

In [0]:
#What is the distribution of eye color among people in the table?
person_df = spark.sql("""select country, eye_color, count(eye_color) as eyecolorcount from person
                            group by country, eye_color
                            order by eyecolorcount desc""")
display(person_df)

country,eye_color,eyecolorcount
USA,BL,4
USA,BR,4
USA,GR,4
India,BL,3
India,BR,1


Inference : We see that maximum people from USA and India have 'BL' (black) as their eye_color followed by 'BR' (brown)

In [0]:
#What is the average age of people in the table? 

age_df = spark.sql("""WITH age AS (
                        SELECT 2023 - YEAR(dob) AS actual_age
                        FROM person
                        )
                     SELECT AVG(actual_age) from age""")
display(age_df)

avg(actual_age)
29.5


Inference : Average age is 29.5

In [0]:
#What is the most common country of origin for people in the table?  
country_count_df = spark.sql("""select count(country) as country_count, country as country_name from person
group by country
order by country_count desc
limit 1""")
display(country_count_df)

country_count,country_name
12,USA


Inference : Most common origin from the data gathered says USA with count 12

In [0]:
#How many unique postal codes exist in the table?  get the postal code and count

postal_df = spark.sql("""select distinct(postal_code) from person""")
display(postal_df)

count_postalcode = spark.sql("""select count(postal_code) from person""")
display(count_postalcode)

postal_code
22356
11111
33333
99999
66666
21056
67890
23456
77777
22222


count(postal_code)
16


Inference : There are 15 postal_codes that are unique and total count is 16

In [0]:
#How many unique cities are there in the table ?

common_city_df = spark.sql("""select distinct(city) from person """)
display(common_city_df)

city
Bangalore
Phoenix
Dallas
Philadelphia
Los Angeles
San Francisco
San Diego
Mumbai
Madikeri
Chicago


Inference : There are 16 unique cities , however the postal code that are unique are only 15, this might need to re-checked. The postal_code data updated for one of the city seems to be wrong(repeated)

Let's see the next dataset that we have which is called favorite_food

In [0]:
create_fav_food_df = spark.read \
                   .option("Inferschema",True) \
                   .option("header",True) \
                   .format("csv") \
                   .load('dbfs:/FileStore/favorite_food.csv')

display(create_fav_food_df)

person_id,food
1,Biryani
1,chicken
1,Fried Chicken
1,Hamburger
1,Pizza
2,Babycorn
2,Ramen
2,Rice
2,Sushi
2,Teriyaki


Here are some of the questions to be answered -
- What is the most popular food among people in the table?
- How many unique foods do people in the table like?
- What is the average number of foods a person likes in the table?
- Are there any people who like the same food?

In [0]:
%sql
DROP TABLE IF EXISTS favfood 

In [0]:
create_fav_food_df.write.mode('overwrite').saveAsTable("favfood")

In [0]:
fav_food_df = spark.sql("""select * from favfood""")
display(fav_food_df)

person_id,food
1,Biryani
1,chicken
1,Fried Chicken
1,Hamburger
1,Pizza
2,Babycorn
2,Ramen
2,Rice
2,Sushi
2,Teriyaki


In [0]:
popular_food = spark.sql("""select food, count(food) as popular from favfood
                            group by food
                            order by popular desc 
                            limit 1""")
display(popular_food)

food,popular
Sushi,4


Inference : Sushi seems to be the most popular food

In [0]:
#How many unique foods do people in the table like?
unique_food = spark.sql("""select distinct(food) from favfood""")
display(unique_food)

total_count = spark.sql("""select count(food) from favfood""")
display(total_count)

food
Salad
Cheeseburger
Biryani
Hamburger
Sushi
Tacos
Steak
Babycorn
Spaghetti
Sandwich


count(food)
36


Inference : There are around 28 food items that are unique and the total food items is 36

In [0]:
#What is the average number of foods a person likes in the table?
avg_liking = spark.sql("""SELECT round(AVG(food_count)) AS avg_num_foods_liked
FROM (
  SELECT person_id, COUNT(food) AS food_count
  FROM favfood
  GROUP BY person_id
) AS person_food_count""")

display(avg_liking)


grouping = spark.sql("""SELECT person_id, COUNT(food) AS food_count
  FROM favfood
  GROUP BY person_id""")


display(grouping)

avg_num_foods_liked
3.0


person_id,food_count
12,2
1,5
13,1
6,2
16,2
3,5
5,3
9,2
4,5
8,1


In [0]:
#Are there any people who like the same food?
people_name = spark.sql("""with people as
                           (select food, count(food) as popular from favfood
                            group by food
                            order by popular desc 
                            limit 1)
                            select p.fname, p.lname, people.food from people
                            join favfood f
                            on people.food = f.food
                            join person p
                            on f.person_id=p.person_id""")
                            
display(people_name)


fname,lname,food
Raju,Desai,Sushi
John,Doe,Sushi
William,Brown,Sushi
Adam,White,Sushi


Inference : Four people Raju, John , William and Adam like Sushi

Let us move on to the next dataset location to answer the following questions -
- What is the most common city of residence for people in the table?
- What is the most common country of origin for people in the table?
- What is the most common nationality among people in the table?
- How many unique countries do people in the table reside in?

In [0]:
location_df = spark.read \
              .option("InferSchema", True) \
              .option("header", True) \
              .format("csv") \
              .load('dbfs:/FileStore/location.csv')
display(location_df)

person_id,city,country,nationality
1,New York,USA,American
2,Tokyo,Japan,Japanese
3,London,UK,British
4,Rome,Italy,Italian
5,Mexico City,Mexico,Mexican
6,Paris,France,French
7,Beijing,China,Chinese
8,Sydney,Australia,Australian
9,Toronto,Canada,Canadian
10,Moscow,Russia,Russian


In [0]:
%sql
DROP TABLE if exists location

In [0]:
location_df.write.mode('overwrite').saveAsTable('location')

In [0]:
%sql
select * from location

person_id,city,country,nationality
1,New York,USA,American
2,Tokyo,Japan,Japanese
3,London,UK,British
4,Rome,Italy,Italian
5,Mexico City,Mexico,Mexican
6,Paris,France,French
7,Beijing,China,Chinese
8,Sydney,Australia,Australian
9,Toronto,Canada,Canadian
10,Moscow,Russia,Russian


In [0]:
#List the unique cities
common_city = spark.sql("""select distinct(city) from location""")
display(common_city)

city
Bangalore
Beijing
Moscow
London
Tokyo
Sydney
Mexico City
Paris
Toronto
Rome


In [0]:
%sql
desc history person

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2023-01-31T11:37:43.000+0000,6342735928450990,rashbeats@gmail.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(1919323896848220),0131-044249-vq9i3hc0,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 16, numOutputBytes -> 3823)",null,Databricks-Runtime/11.3.x-scala2.12


In [0]:
%sql
select * from person;

person_id,fname,lname,eye_color,dob,street,city,state,country,postal_code
1,Chandru,Rathod,BR,1995-02-23,H1 road,Mumbai,MH,India,23456
2,Raju,Desai,BL,1993-01-30,Udagiri,Bangalore,KA,India,22356
3,Rajendra,Diman,BL,1993-01-30,Kengeri,Madikeri,KA,India,21056
4,Rigan,Thomas,BL,1993-01-30,Udagiri,Hassan,KA,India,22056
5,John,Doe,BL,1999-01-01,123 Main St,San Francisco,California,USA,12345
6,Jane,Smith,GR,1998-02-14,456 Oak Ave,Los Angeles,California,USA,67890
7,Michael,Johnson,BR,1997-03-25,789 Maple St,New York,New York,USA,11111
8,Emily,Davis,BL,1996-04-16,111 Pine St,Seattle,Washington,USA,22222
9,William,Brown,BR,1995-05-27,222 Cedar Ave,Chicago,Illinois,USA,33333
10,Brittney,Thomas,BL,1990-10-12,777 Maple St,Phoenix,Arizona,USA,88888


In [0]:

%sql
INSERT INTO person 
    VALUES (17, 'Jason', 'Tom', 'BR', '1991-09-11', '666 Cedar Ave', 'Bangalore', 'KA', 'India', '22356'),
           (18, 'Rami', 'Sai', 'BL', '1990-09-11', 'Prestige city', 'Bangalore', 'KA', 'India', '22356'),
           (19, 'Sai', 'Krishna', 'BL', '1991-10-11', 'New town', 'Bangalore', 'KA', 'India', '22356');

num_affected_rows,num_inserted_rows
3,3


In [0]:
%sql
INSERT INTO person 
    VALUES (22, 'Santoshi', 'Rane', 'BR', '1996-07-11', 'Nagawara', 'Bangalore', 'KA', 'India', '22356'),
            (22, 'Amit', 'Mittal', 'BR', '1992-09-11', 'Nagawara', 'Bangalore', 'KA', 'India', '22356'),
           (23, 'Sonu', 'Raghav', 'BR', '1991-10-11', 'Ecity', 'Bangalore', 'KA', 'India', '22356'),
           (24, 'Rajeev', 'Dixit', 'BR', '1990-09-11', 'Yelahanka', 'Bangalore', 'KA', 'India', '22356'),
           (25, 'Janvi', 'Bindra', 'BR', '1991-09-11', 'Cedar Ave', 'Bangalore', 'KA', 'India', '22356')

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql
UPDATE PERSON 
SET person_id = 21
where fname = 'Santoshi'

num_affected_rows
1


In [0]:
%sql
select * from person

person_id,fname,lname,eye_color,dob,street,city,state,country,postal_code
1,Chandru,Rathod,BR,1995-02-23,H1 road,Mumbai,MH,India,23456
2,Raju,Desai,BL,1993-01-30,Udagiri,Bangalore,KA,India,22356
3,Rajendra,Diman,BL,1993-01-30,Kengeri,Madikeri,KA,India,21056
4,Rigan,Thomas,BL,1993-01-30,Udagiri,Hassan,KA,India,22056
5,John,Doe,BL,1999-01-01,123 Main St,San Francisco,California,USA,12345
6,Jane,Smith,GR,1998-02-14,456 Oak Ave,Los Angeles,California,USA,67890
7,Michael,Johnson,BR,1997-03-25,789 Maple St,New York,New York,USA,11111
8,Emily,Davis,BL,1996-04-16,111 Pine St,Seattle,Washington,USA,22222
9,William,Brown,BR,1995-05-27,222 Cedar Ave,Chicago,Illinois,USA,33333
10,Brittney,Thomas,BL,1990-10-12,777 Maple St,Phoenix,Arizona,USA,88888


In [0]:
#What is the most common city of residence for people in the table?
common_city = spark.sql("""
SELECT city, COUNT(distinct(person_id)) as count
FROM person
GROUP BY city
order by count desc limit 1
""")

display(common_city)

city,count
Bangalore,10


In [0]:
#What is the most common country of origin for people in the table?
#What is the most common nationality among people in the table?

common_origin = spark.sql("""with origin as
(SELECT p.city, COUNT(distinct(p.person_id)) as count
FROM person p
GROUP BY 1
order by count desc limit 1)
select origin.city, origin.count, l.nationality, l.country from origin
join location l
on origin.city=l.city
""")
display(common_origin)

city,count,nationality,country
Bangalore,10,Indian,India


In [0]:
#How many unique countries do people in the table reside in?

unique_countries = spark.sql("""select distinct(country) from location""")
display(unique_countries)

country
Russia
France
China
India
Italy
USA
Mexico
UK
Canada
Japan


Let us move on to the fourth dataset that is person_info and answer following questions

- What is the average salary of employees in the table?
- What is the most common department among employees in the table?
- What is the average salary in each department?
- What is the most common company name in the table?

In [0]:
personinfo_df = spark.read \
                .option("InferSchema",True) \
                .option("header",True) \
                .format("csv") \
                .load('dbfs:/FileStore/personinfo.csv')

In [0]:
personinfo_df.write.mode('overwrite').saveAsTable("salary")

In [0]:
%sql
select * from salary

person_id,salary,department,company_name
1,50000,Finance,ABC Inc.
2,55000,Marketing,ABC Inc.
3,60000,IT,XYZ Ltd.
4,65000,HR,XYZ Ltd.
5,70000,Finance,XYZ Ltd.
6,50000,IT,SKL Inc.
7,80000,IT,C Corp Inc.
8,16000,IT,Atos Inc.
9,20000,IT,ABC Inc.
10,24000,IT,C Corp Inc.


In [0]:
#Find the average salary
avg_salary = spark.sql("""select avg(salary) from salary""")
display(avg_salary)

avg(salary)
44480.0


In [0]:
#Get the count of all departments and find the Most common department
common_dept = spark.sql("""select department, count(department) as count
                          from salary
                          group by 1
                          order by count desc
                          """)
display(common_dept)

department,count
IT,21
Finance,2
HR,1
Marketing,1


Inference : The IT department is the common with count 21

In [0]:
#What is the most common company name in the table?
common_company = spark.sql("""select company_name, count(company_name) as count
                          from salary
                          group by 1
                          order by count desc limit 1
                          """)
display(common_company)

company_name,count
ABC Inc.,6


In [0]:
#Get the list of all companies and the count of each in the table
list_company = spark.sql("""select company_name, count(company_name) as count
                          from salary
                          group by 1
                          order by count desc
                          """)
display(list_company)

company_name,count
ABC Inc.,6
Top Inc.,5
C Corp Inc.,4
XYZ Ltd.,3
TMS Inc.,2
TTC Inc.,1
Sal Inc.,1
SKL Inc.,1
Atos Inc.,1
ITK Inc.,1


Now let us answer the remaining questions in the list -
- What is the average salary of people living in a particular city or country?
- What is the most common eye color among people in a specific department of a company?
- Which country has the most number of people with a specific favorite food?
- What is the average age of people in a specific department of a company?
- How many people of different nationalities work for a specific company?
- Which city has the highest number of people with a specific eye color?
- What is the most common favorite food among people in a specific country?
- What is the average salary of people with a specific eye color in a specific city?
- How many people from a specific state work for a company located in a specific country?
- What is the average salary of people with a specific nationality working for a specific company?

In [0]:
#What is the average salary of people living in a particular city or country?
avg_sal_city = spark.sql("""select avg(s.salary), p.city 
                            from person p
                            join
                            salary s
                            on p.person_id = s.person_id
                            group by p.city
                            order by 1 """)
display(avg_sal_city)

avg(salary),city
16000.0,Seattle
20000.0,Chicago
24000.0,Phoenix
26000.0,Houston
36000.0,Philadelphia
38900.0,Bangalore
39000.0,San Diego
46000.0,Miami
50000.0,Los Angeles
50000.0,Mumbai


In [0]:
#What is the average salary of people with a specific nationality working for a specific company?

avg_sal_nat_com= spark.sql("""select avg(s.salary), l.nationality, s.company_name 
                            from location l
                            join
                            salary s
                            on l.person_id = s.person_id
                            group by 2,3
                            order by 1 """)
display(avg_sal_nat_com)

avg(salary),nationality,company_name
16000.0,Australian,Atos Inc.
20000.0,Canadian,ABC Inc.
24000.0,Russian,C Corp Inc.
39000.0,Indian,Top Inc.
50000.0,French,SKL Inc.
50000.0,American,ABC Inc.
55000.0,Japanese,ABC Inc.
60000.0,British,XYZ Ltd.
65000.0,Italian,XYZ Ltd.
70000.0,Mexican,XYZ Ltd.


In [0]:
#What is the most common eye color among people in a specific department of a company?
common_eye_color = spark.sql("""select s.department, p.eye_color, count(p.eye_color) as count
                                from person p
                                join salary s
                                on p.person_id=s.person_id
                                group by s.department, p.eye_color
                                order by count desc
                                """)
display(common_eye_color)

department,eye_color,count
IT,BR,11
IT,BL,6
IT,GR,4
Marketing,BL,1
Finance,BL,1
HR,BL,1
Finance,BR,1


In [0]:
#Which country has the most number of people with a specific favorite food?
#What is the most common favorite food among people in a specific country?

fav_food_country = spark.sql("""select p.country, f.food, count(f.food)
                               from person p
                               join favfood f
                               on p.person_id = f.person_id
                               group by 1,2
                               order by 3 desc
                               """)

display(fav_food_country)

country,food,count(food)
USA,Sushi,3
USA,Tacos,2
India,Steak,2
India,Hamburger,2
USA,Lobster,1
India,Teriyaki,1
USA,Pastry,1
India,Spaghetti,1
India,Rice,1
India,Lasagna,1


Inference : The Sushi food is most favorite food item with 3 voting for it in USA followed by Tacos, Steak and Hamburger in USA and India

In [0]:
#What is the average age of people in a specific department of a company?
avg_age_dept = spark.sql("""with avgage
                             as (select (2023-year(dob)) as age, person_id from person )
                             select s.department, round(avg(a.age)) from avgage a
                             join salary s
                             on a.person_id=s.person_id
                             group by department
                             order by 2
                             """)

display(avg_age_dept)

department,"round(avg(age), 0)"
Finance,26.0
HR,30.0
Marketing,30.0
IT,31.0


In [0]:
#How many people of different nationalities work for a specific company?
nat_comp = spark.sql("""select  s.company_name, l.nationality, count(l.nationality) from location l
                           join salary s
                           on l.person_id = s.person_id
                           group by 1,2
                           order by 3 desc
                           """)

display(nat_comp)

company_name,nationality,count(nationality)
XYZ Ltd.,Italian,1
ABC Inc.,American,1
XYZ Ltd.,Mexican,1
C Corp Inc.,Chinese,1
Top Inc.,Indian,1
Atos Inc.,Australian,1
SKL Inc.,French,1
C Corp Inc.,Russian,1
ABC Inc.,Japanese,1
ABC Inc.,Canadian,1


In [0]:
#Which city has the highest number of people with a specific eye color?

city_eye_max = spark.sql("""select city, eye_color, count(eye_color) as eyecolorcount from person
                            group by city, eye_color
                            order by eyecolorcount desc""")
display(city_eye_max)

city,eye_color,eyecolorcount
Bangalore,BR,7
Bangalore,BL,3
Los Angeles,GR,1
San Francisco,BL,1
Mumbai,BR,1
Dallas,BR,1
Seattle,BL,1
Houston,GR,1
Madikeri,BL,1
New York,BR,1


In [0]:
#What is the average salary of people with a specific eye color (say BL) in a specific city say 'Bangalore'?

city_eye_max = spark.sql("""select p.city, p.eye_color, round(avg(s.salary)) as avgsal from person p
                            join salary s
                            on p.person_id = s.person_id
                            where p.eye_color = 'BL' and p.city = 'Bangalore'
                            group by p.city, p.eye_color
                            order by avgsal desc""")
display(city_eye_max)

city,eye_color,avgsal
Bangalore,BL,45667.0


In [0]:
#How many people from a specific state work for a company located in a specific country?

state_country = spark.sql("""select p.state, s.company_name , p.country, count(p.person_id)
                             from person p
                             join salary s
                              on p.person_id= s.person_id
                              group by 1,2,3
                              order by 4 desc""")

display(state_country)

state,company_name,country,count(person_id)
KA,ABC Inc.,India,3
Texas,Top Inc.,USA,2
KA,XYZ Ltd.,India,2
KA,TMS Inc.,India,2
KA,C Corp Inc.,India,2
Illinois,ABC Inc.,USA,1
Washington,Atos Inc.,USA,1
Florida,Top Inc.,USA,1
New York,C Corp Inc.,USA,1
Arizona,C Corp Inc.,USA,1


Inference : The highest number of people working for ABC Inc, India are from Karnataka